# Categorical Variables - Exercise

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
X=pd.read_csv('train.csv', index_col='Id')
X_test=pd.read_csv('test.csv', index_col='Id')

First, we need to remove rows with missing target and then separate target from predictors.

In [3]:
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
##
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

In [4]:
# for simplier exercise we can drop columns with missing values

In [5]:
cols_with_missing=[col for col in X.columns if X[col].isnull().any()]

In [6]:
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [8]:
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


For scoring different approaches we will create function score_dataset

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [10]:
def score_dataset (X_train,X_valid,y_train, y_valid):
    model=RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train,y_train)
    pred=model.predict(X_valid)
    return mean_absolute_error(y_valid, pred)

# 1. Approach - drop categorical data

In [11]:
drop_X_train=X_train.select_dtypes(exclude=['object'])
drop_X_valid=X_valid.select_dtypes(exclude=['object'])

In [12]:
print('MAE for Dropping categorical var: ')
print(score_dataset(drop_X_train,drop_X_valid, y_train, y_valid))

MAE for Dropping categorical var: 
17837.82570776256


# 2. Label Encoding

In [13]:
# finding catgorical columns

In [14]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

In [15]:
# columns that can be safely encoded ( if there is too many distinct observation)
good_label_cols=[col for col in object_cols if set(X_train[col])==set(X_valid[col])]

In [16]:
bad_label_cols=list(set(object_cols)-set(good_label_cols))

In [17]:
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'ExterQual', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Condition1', 'HeatingQC', 'Exterior2nd', 'SaleType', 'Foundation', 'Neighborhood', 'RoofStyle', 'Condition2', 'ExterCond', 'Functional', 'Exterior1st', 'RoofMatl', 'Utilities', 'LandSlope', 'Heating']


If you now write code to: 
- fit a label encoder to the training data, and then 
- use it to transform both the training and validation data, 

you'll get an error.  Can you see why this is the case?  

This is a common problem that you'll encounter with real-world data, and there are many approaches to fixing this issue.  For instance, you can write a custom label encoder to deal with new categories.  The simplest approach, however, is to drop the problematic categorical columns.  

Run the code cell below to save the problematic columns to a Python list `bad_label_cols`.  Likewise, columns that can be safely label encoded are stored in `good_label_cols`.

In [18]:
# first we are dopping bad columns

In [19]:
label_X_train=X_train.drop(bad_label_cols, axis=1)
label_X_valid=X_valid.drop(bad_label_cols, axis=1)

In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoder=LabelEncoder()
for col in set(good_label_cols):
    label_X_train[col]=label_encoder.fit_transform(X_train[col])
    label_X_valid[col]=label_encoder.transform(X_valid[col])

In [21]:
print('MAE for Label Encoder: ')
print(score_dataset(label_X_train,label_X_valid, y_train, y_valid))

MAE for Label Encoder: 
17575.291883561644


Better approach then dropping columns

# 3.Step :Investing Cardinality

So far, we've tried two different approaches to dealing with categorical variables.  And, we've seen that encoding categorical data yields better results than removing columns from the dataset.

Soon, we'll try one-hot encoding.  Before then, there's one additional topic we need to cover.

In [22]:
# we want to find unique entries for each column with categorical data

In [23]:
object_unique=list(map(lambda col: X_train[col].nunique(), object_cols))

The map() function executes a specified function for each item in a iterable. The item is sent to the function as a parameter.

In [24]:
d=dict(zip(object_cols, object_unique))

In [25]:
#print number of unique entries by column in ascending order

In [26]:
sorted(d.items(), key=lambda x:x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

We refer to the number of unique entries of a categorical variable as the **cardinality** of that categorical variable.  For instance, the `'Street'` variable has cardinality 2.

For large datasets with many rows, one-hot encoding can greatly expand the size of the dataset.  For this reason, we typically will only one-hot encode columns with relatively low cardinality.  Then, high cardinality columns can either be dropped from the dataset, or we can use label encoding.



# 3. Approach - One-hot Encoding

We will encode only columns with low cardinality columns

In [27]:
low_cardinality_cols=[col for col in object_cols if X_train[col].nunique() < 10]

This columns will remain in dataset because they have cardinality smaller then 10. Know we are finding columns that will be removed from data.

In [28]:
high_cardinality_cols=list(set(object_cols)-set(low_cardinality_cols))

In [29]:
print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Exterior2nd', 'Exterior1st', 'Neighborhood']


In [30]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder=OneHotEncoder(handle_unknown='ignore', sparse=False)

In [31]:
OH_cols_train=pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid=pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

In [32]:
OH_cols_train.index=X_train.index
OH_cols_valid.index=X_valid.index

In [33]:
num_X_train=X_train.drop(object_cols, axis=1)
num_X_valid=X_valid.drop(object_cols, axis=1)

In [35]:
OH_X_train=pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid=pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [36]:
print('MAE for OH Encoder: ')
print(score_dataset(OH_X_train,OH_X_valid, y_train, y_valid))

MAE for OH Encoder: 
17525.345719178084


In [37]:
# better result!!!

Process:
1. Separate low cardinality columns from other object_colums
2. Define OneHot Encoder
3. implement it on low cardinality columns
4. return index
5. find numeric data, as dropping object data
6. connect numeric data and transformed data
7. evalute MAE